In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import Chroma, FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import GoogleGenerativeAI

d:\SOURAV\GenAI\HotelChatBot\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
llm =  GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

In [4]:
loader=PyPDFLoader("knowledge.pdf")
docs = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024)
documents = text_splitter.split_documents(docs)

In [5]:
PERSIST_STORAGE = "./storage"

In [6]:
chroma_db = Chroma.from_documents(documents=documents, embedding= embeddings, persist_directory=PERSIST_STORAGE)
faiss_db = FAISS.from_documents(documents=documents, embedding= embeddings)
faiss_db.save_local("faiss_index")
faiss_db=FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)

In [7]:
retriever = faiss_db.as_retriever()
ret = chroma_db.as_retriever()
result = retriever.invoke("Can I extend my stay?")
res = ret.invoke("Can I extend my stay?")

In [8]:
print(result[0].page_content)

109. The TIC service guarantee will not be applicable for any service delivery failure at the hotel, the 
member will have to liaise with the hotel for the same. The TIC team will help the member find a 
resolution for the service concern raised, however hot el related service issues will not be covered under 
the TIC service guarantee.


In [9]:
print(res[0].page_content)

109. The TIC service guarantee will not be applicable for any service delivery failure at the hotel, the 
member will have to liaise with the hotel for the same. The TIC team will help the member find a 
resolution for the service concern raised, however hot el related service issues will not be covered under 
the TIC service guarantee.


In [10]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="hotel_search",
    description="Search for information about Hotel check-in, check-out, facilities. \
        For any questions about the hotel policies, you must use this tool!",
)

In [11]:
TAVILY_API_KEY

'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'

In [12]:
search = TavilySearchResults()

In [13]:
search.dict()

{'name': 'tavily_search_results_json',
 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
 'args_schema': langchain_community.tools.tavily_search.tool.TavilyInput,
 'return_direct': False,
 'verbose': False,
 'tags': None,
 'metadata': None,
 'handle_tool_error': False,
 'handle_validation_error': False,
 'api_wrapper': {'tavily_api_key': SecretStr('**********')},
 'max_results': 5}

In [14]:
tools = [search, retriever_tool]

# Agent

In [15]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [16]:
prompt = hub.pull("hwchase17/react")

ImportError: Could not import langchainhub, please install with `pip install langchainhub`.